In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('titanic.csv')

In [ ]:
# Create an in-memory SQLite database
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [ ]:
df.to_sql('titanic', con=engine)

In [4]:
import psycopg2 as pg
pg.connect


import pandas as pd
import psycopg2 as pg


# Posgresql database parameters
dbname ='hartabvb'
user = 'hartabvb'
password = '9jk1ptYGv0qclPjUQybvhF8bnmAXPmNv'
host = 'stampy.db.elephantsql.com'

pg_conn = pg.connect(dbname=dbname, user=user, password=password, host=host)

In [5]:
def copy_table(table_name, csv_file):
    
    import pandas as pd
    
    # Converting csv file into Pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create an in-memory SQLite database
    from sqlalchemy import create_engine
    engine = create_engine('sqlite://', echo=False)
    df.to_sql('titanic', con=engine, index=False)
    
    # All rows as tuples from new SQL table
    results = engine.execute("SELECT * FROM titanic").fetchall()

    
    # Cursor
    pg_curs = pg_conn.cursor()
    
    # dataframe of columns and types
    convert_types ={'int64':'int', 'object': 'text', 'float64': 'real'}
    
    col_types = pd.DataFrame(df.dtypes, columns=['types'])
    col_types = col_types.replace(convert_types)
    
    # Creating string of column names and type for creating SQL table
    col_n_type = []
    for i, j in zip(col_types.index, col_types.types.values):
        result = i + ' ' + j
        col_n_type.append(result)
    
    # Converting into one string
    col_n_type = ','.join(col_n_type).replace('/', '_').replace(' Aboard', '_Aboard')
    
    # Creating SQL table
    create = f"CREATE TABLE {table_name}({col_n_type});"
    
    pg_curs.execute(create)

    # Now we insert the data in the empty table created
    col_names = ','.join(df.columns).replace('/', '_').replace(' Aboard', '_Aboard')
    rows = df.values
    
    for result in results:
        insert_result = f"INSERT INTO {table_name} ({col_names}) VALUES" + str(result)
        pg_curs.execute(insert_result)


    #pg_conn.commit()

In [6]:
copy_table('titanic', 'titanic.csv')

ProgrammingError: column "Miss. Ellen O'Dwyer" does not exist
LINE 1: ...Aboard,Parents_Children_Aboard,Fare) VALUES(1, 3, "Miss. Ell...
                                                             ^
